# Spark Streaming

Kode di bawah ini bertujuan untuk melakukan penghitungan kata secara real-time dari data streaming yang diterima melalui socket.

Spark Streaming yang membaca teks dari socket, memecahnya menjadi kata-kata, menghitung jumlah kemunculan setiap kata, dan menampilkan hasilnya ke konsol. Aplikasi ini sangat berguna untuk pemrosesan data real-time, seperti menghitung kata dalam streaming teks yang terus-menerus.

Sebelum menjalankan harap dipastikan port 8080 sudah standby menerima koneksi.
Perintah: `nc -lk 8080`.

Untuk melakukan bisa membuka File > New > Terminal.

File ini adalah penjelasan baris per baris dari file `08-spark-streaming.py`.

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()

* SparkSession.builder.appName("NetworkWordCount").getOrCreate(): Baris ini menginisialisasi SparkSession, yang merupakan entry point utama untuk API DataFrame dan Dataset. Di sini, aplikasi Spark dinamai "NetworkWordCount".

In [ ]:
# Membaca data streaming dari socket
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 8080).load()

* spark.readStream.format("socket"): Menentukan bahwa data streaming akan dibaca dari socket.
* option("host", "localhost").option("port", 9999): Menentukan bahwa Spark harus terhubung ke host lokal (localhost) pada port 9999 untuk menerima data streaming.
* load(): Memulai pembacaan data dari socket.
* Pada tahap ini, setiap baris teks yang dikirim ke socket ini akan dibaca sebagai satu baris dalam DataFrame lines.

In [ ]:
# Split setiap baris menjadi kata-kata
words = lines.selectExpr("explode(split(value, ' ')) as word")

* split(value, ' '): Memecah setiap baris teks (value) menjadi array kata-kata berdasarkan pemisah spasi (' ').
* explode(split(value, ' ')): Mengubah array kata-kata tersebut menjadi baris-baris terpisah, satu kata per baris.
* selectExpr("explode(split(value, ' ')) as word"): Menyimpan hasilnya dalam DataFrame words, di mana setiap baris berisi satu kata.

In [ ]:
# Hitung jumlah kemunculan setiap kata
wordCounts = words.groupBy("word").count()

* words.groupBy("word").count(): Mengelompokkan kata-kata yang sama dan menghitung berapa kali masing-masing kata muncul dalam data streaming.

In [ ]:
# Mulai query untuk menulis hasil ke konsol
query = wordCounts.writeStream.outputMode("complete").format("console").start()

* wordCounts.writeStream: Mengambil DataFrame wordCounts dan menulis hasilnya sebagai stream.
* outputMode("complete"): Menentukan bahwa seluruh tabel hasil (jumlah setiap kata) akan ditampilkan setiap kali ada perubahan.
* format("console"): Menentukan bahwa hasilnya akan ditulis ke konsol.
* start(): Memulai query streaming.

In [ ]:
query.awaitTermination()

* query.awaitTermination(): Membuat aplikasi tetap berjalan dan menunggu hingga streaming dihentikan secara manual.